<a href="https://colab.research.google.com/github/stevehooker/welsh-dragons/blob/main/notebooks/comfyui_colab_with_manager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ComfyUI on Google Colab

This notebook sets up ComfyUI with Manager on Google Colab.

**Features:**
- Google Drive integration for persistent storage
- ComfyUI Manager for easy custom node management
- Cloudflare tunnel for external access
- Automatic dependency installation

## 1. Mount Google Drive

Mount your Google Drive to save ComfyUI installation and models persistently.

In [ ]:
#@title Mount Google Drive
#@markdown This will save your ComfyUI installation to Google Drive

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}

if USE_GOOGLE_DRIVE:
    print("Mounting Google Drive...")
    from google.colab import drive
    drive.mount('/content/drive')
    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive
else:
    WORKSPACE = "/content/ComfyUI"
    %cd /content

print(f"Workspace: {WORKSPACE}")

## 2. Install ComfyUI

Clone ComfyUI repository (only runs on first setup) and optionally update it.

In [ ]:
#@title Install/Update ComfyUI
#@markdown Update ComfyUI to latest version?

UPDATE_COMFY_UI = True  #@param {type:"boolean"}

# Clone ComfyUI if it doesn't exist
![ ! -d {WORKSPACE} ] && echo "=== Installing ComfyUI ===" && git clone https://github.com/comfyanonymous/ComfyUI {WORKSPACE}

%cd {WORKSPACE}

# Update ComfyUI if requested
if UPDATE_COMFY_UI:
    print("=== Updating ComfyUI ===")
    
    # Fix permissions for Google Drive files
    ![ -f ".ci/nightly/update_windows/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/nightly/update_windows/update_comfyui_and_python_dependencies.bat
    ![ -f ".ci/nightly/windows_base_files/run_nvidia_gpu.bat" ] && chmod 755 .ci/nightly/windows_base_files/run_nvidia_gpu.bat
    ![ -f ".ci/update_windows/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/update_windows/update_comfyui_and_python_dependencies.bat
    ![ -f ".ci/update_windows_cu118/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/update_windows_cu118/update_comfyui_and_python_dependencies.bat
    ![ -f ".ci/update_windows/update.py" ] && chmod 755 .ci/update_windows/update.py
    ![ -f ".ci/update_windows/update_comfyui.bat" ] && chmod 755 .ci/update_windows/update_comfyui.bat
    ![ -f ".ci/update_windows/README_VERY_IMPORTANT.txt" ] && chmod 755 .ci/update_windows/README_VERY_IMPORTANT.txt
    ![ -f ".ci/update_windows/run_cpu.bat" ] && chmod 755 .ci/update_windows/run_cpu.bat
    ![ -f ".ci/update_windows/run_nvidia_gpu.bat" ] && chmod 755 .ci/update_windows/run_nvidia_gpu.bat
    
    # Checkout main/master branch and pull latest
    !git checkout master 2>/dev/null || git checkout main 2>/dev/null || true
    !git pull

print("ComfyUI installation complete!")

## 3. Install Dependencies

Install Python packages required for ComfyUI. Skip this if already installed.

In [ ]:
#@title Install Dependencies
#@markdown Install Python packages? (Uncheck to skip if already installed)

INSTALL_DEPENDENCIES = True  #@param {type:"boolean"}

if INSTALL_DEPENDENCIES:
    print("=== Installing Dependencies ===")
    
    # Core dependencies
    !pip3 install -q accelerate
    !pip3 install -q einops transformers>=4.28.1 safetensors>=0.4.2 aiohttp pyyaml Pillow scipy tqdm psutil tokenizers>=0.13.3
    
    # PyTorch with CUDA 12.1
    !pip3 install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
    
    # Additional requirements
    !pip3 install -q torchsde
    !pip3 install -q kornia>=0.7.1 spandrel soundfile sentencepiece
    
    # Video support
    !pip3 install -q av
    
    # ComfyUI requirements (includes workflow templates and docs)
    !pip3 install -q -r requirements.txt
    
    print("Dependencies installed!")
else:
    print("Skipping dependency installation")

## 4. Install ComfyUI Manager

ComfyUI Manager makes it easy to install custom nodes and models.

In [ ]:
#@title Install ComfyUI Manager
#@markdown Install ComfyUI Manager for easier custom node management?

USE_COMFYUI_MANAGER = True  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}

if USE_COMFYUI_MANAGER:
    %cd {WORKSPACE}/custom_nodes
    
    # Fix permissions for Google Drive files
    ![ -f "ComfyUI-Manager/check.sh" ] && chmod 755 ComfyUI-Manager/check.sh
    ![ -f "ComfyUI-Manager/scan.sh" ] && chmod 755 ComfyUI-Manager/scan.sh
    ![ -f "ComfyUI-Manager/node_db/dev/scan.sh" ] && chmod 755 ComfyUI-Manager/node_db/dev/scan.sh
    ![ -f "ComfyUI-Manager/node_db/tutorial/scan.sh" ] && chmod 755 ComfyUI-Manager/node_db/tutorial/scan.sh
    ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh
    ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-win.bat" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-win.bat
    
    # Clone Manager if it doesn't exist
    ![ ! -d ComfyUI-Manager ] && echo "=== Installing ComfyUI-Manager ===" && git clone https://github.com/ltdrdata/ComfyUI-Manager
    
    # Update Manager
    %cd ComfyUI-Manager
    !git pull
    
    %cd {WORKSPACE}
    
    # Install custom node dependencies
    if INSTALL_CUSTOM_NODES_DEPENDENCIES:
        print("=== Installing Custom Node Dependencies ===")
        !pip install -q GitPython
        !python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies
    
    print("ComfyUI Manager installed!")
else:
    print("Skipping ComfyUI Manager installation")

## 5. Start ComfyUI

Launch ComfyUI server with cloudflared tunnel for external access.

**Click the URL that appears below to access ComfyUI!**

In [ ]:
# Download and install cloudflared
!wget -q -P ~ https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket

def iframe_thread(port):
    """Wait for ComfyUI to start, then launch cloudflared tunnel"""
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()
    
    print("\n" + "=" * 60)
    print("ComfyUI is ready! Starting cloudflared tunnel...")
    print("=" * 60 + "\n")
    
    p = subprocess.Popen(
        ["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    
    for line in p.stderr:
        l = line.decode()
        if "trycloudflare.com" in l:
            print("\n" + "=" * 60)
            print("ACCESS COMFYUI HERE:")
            print(l[l.find("http"):].strip())
            print("=" * 60 + "\n")

# Start cloudflared in background thread
threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

# Start ComfyUI
print("Starting ComfyUI server...")
%cd {WORKSPACE}
!python main.py --dont-print-server